<a href="https://colab.research.google.com/github/rayafintech2020/RAG/blob/main/LlamaIndex_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

At its core, LlamaIndex contains a toolkit designed to easily connect LLM’s with your external data.





1.   Creating and Quering Index
2.   Saving and Loading Index
3.   Customize LLM
4.   Customize Prompt
5.   Customize Embedding


In [53]:
!pip install llama-index pypdf sentence_transformers -q

By default, we use the OpenAI GPT-3 text-davinci-003 model.

https://gpt-index.readthedocs.io/en/latest/getting_started/installation.html

In [54]:
%pip install llama-index-llms-openrouter

In [59]:
from llama_index.llms.openrouter import OpenRouter


llm = OpenRouter(
    api_key="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",
    max_tokens=256,
    context_window=4096,
    model="deepseek/deepseek-chat:free",
)

In [60]:
from llama_index.core.llms import ChatMessage

message = ChatMessage(role="user", content="can you write a short story?")
resp = llm.chat([message])
print(resp)

assistant: **The Whispering Woods**

In the heart of the Whispering Woods, where the trees stood tall and ancient, there was a legend. It was said that the forest held a secret—a hidden glade where the trees could speak. But only those pure of heart could hear their voices.

One crisp autumn morning, a young girl named Elara ventured into the woods. She had always been drawn to the forest, its rustling leaves and dappled sunlight calling to her like an old friend. Elara carried a small satchel with her, filled with bread and cheese, and a curious heart.

As she walked deeper into the woods, the air grew cooler, and the light filtered through the canopy in golden shafts. She felt a strange pull, as if the forest itself was guiding her. After what felt like hours, she stumbled upon a glade she had never seen before. It was a perfect circle, the grass soft and green, and in the center stood an ancient oak, its bark gnarled and wise.

Elara approached the tree, her heart pounding. She plac

##Creating Index

https://gpt-index.readthedocs.io/en/latest/guides/primer/index_guide.html

In [36]:
!pip install pypdf2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.7 MB/s eta 0:00:00


In [62]:
%pip install llama-index-embeddings-huggingface
%pip install llama-index-embeddings-instructor

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 4.5 MB/s eta 0:00:00
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 3.4.1
    Uninstalling sentence-transformers-3.4.1:
      Successfully uninstalled sentence-transformers-3.4.1


In [68]:
from llama_index.core import Document, VectorStoreIndex
from llama_index.llms.openrouter import OpenRouter
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
import PyPDF2

# باز کردن فایل PDF و استخراج متن
with open('/content/Mafahim _Kamel_.pdf', 'rb') as file:
    reader = PyPDF2.PdfReader(file)
    text = ''
    for page in reader.pages:
        text += page.extract_text()

# تبدیل متن استخراج شده به یک شیء Document
documents = [Document(text=text)]

# بارگذاری مدل Embedding از Hugging Face
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

# تعریف OpenRouter LLM
llm = OpenRouter(
    api_key="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",
    max_tokens=256,
    context_window=4096,
    model="deepseek/deepseek-chat:free",
)

# ایجاد ایندکس با استفاده از Embedding سفارشی
index = VectorStoreIndex.from_documents(documents, embed_model=embed_model)

# ساخت query_engine با LLM سفارشی OpenRouter
query_engine = index.as_query_engine(llm=llm)

# آزمون پرسش
response = query_engine.query("این PDF درباره چیست؟")
print(response)


این PDF به موضوعات مالی و حسابداری می‌پردازد. در آن به شناسایی عناصر مالی، تغییرات در دارایی‌ها و بدهی‌ها، و اندازه‌گیری مقادیر پولی اشاره شده است. همچنین، نظریه‌های اقتصادی کلاسیک در مورد درآمد، مصرف، سرمایه و ذخیره‌سازی ثروت مورد بررسی قرار گرفته‌اند. این متن به اهمیت شواهد کافی برای وقوع تغییرات مالی و قابلیت اندازه‌گیری مقادیر پولی تأکید می‌کند.


In [69]:
response = query_engine.query("بر اساس این فایل منافع اقتصادی آتی چیست؟")
print(response)

منافع اقتصادی آتی به مزایا و سودهایی اشاره دارد که در آینده از طریق سرمایه‌گذاری یا ذخیره‌سازی منابع به دست می‌آید. این منافع می‌تواند شامل درآمدهای آینده، افزایش ظرفیت تولیدی یا بهبود رفاه اقتصادی باشد. به عبارت دیگر، سرمایه‌گذاری‌های امروز می‌توانند منجر به دستاوردهای مالی و اقتصادی در آینده شوند.


##Saving and Loading Index

In [74]:
index.storage_context.persist("naval_index")

In [73]:
import os
print(os.getcwd())


/content


In [82]:
from llama_index.core import StorageContext, load_index_from_storage
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# تعریف مدل Embedding
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

# بازسازی context ذخیره‌شده
storage_context = StorageContext.from_defaults(persist_dir="naval_index")

# بارگذاری ایندکس
new_index = load_index_from_storage(storage_context, embed_model=embed_model)


In [83]:
new_query_engine = new_index.as_query_engine(llm=llm)
response = new_query_engine.query("متن در مورد چه بود؟")
print(response)

متن در مورد ارزیابی عملکرد و انعطاف‌پذیری مالی در یک واحد تجاری بود. این متن به بررسی چگونگی کاهش هزینه‌ها و افزایش درآمد از طریق مدیریت منابع مالی و دارایی‌ها پرداخته است. همچنین، به موضوع انعطاف‌پذیری مالی اشاره کرده که شامل از دست دادن برخی منافع برای کسب مزایای دیگر است، مانند نگهداری دارایی‌هایی که به راحتی قابل فروش هستند. این رویکرد ممکن است شامل کاهش نرخ بازدهی در مقایسه با سرمایه‌گذاری‌های دیگر باشد.


##Customizing LLM's

https://gpt-index.readthedocs.io/en/latest/how_to/customization/service_context.html

In [ ]:
from llama_index import LLMPredictor, ServiceContext

from langchain.chat_models import ChatOpenAI

llm_predictor = LLMPredictor(llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo"))


service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor)


custom_llm_index = VectorStoreIndex.from_documents(
    documents, service_context=service_context
)

In [ ]:
custom_llm_query_engine = custom_llm_index.as_query_engine()
response = custom_llm_query_engine.query("who is this text about?")
print(response)

This text is about Naval Ravikant.


##Custom Prompt

In [85]:
from llama_index.core import Prompt

template = (
    "We have provided context information below. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
    "برای هر سوال 5 مورد مهم نام ببر: {query_str}\n"
)
qa_template = Prompt(template)

In [88]:
query_engine = new_index.as_query_engine(llm=llm , text_qa_template=qa_template)
response = query_engine.query("نحوه ارزش گذاری به چند شکل است؟")
print(response)

برای پاسخ به سوال "نحوه ارزش‌گذاری به چند شکل است؟" می‌توان به موارد زیر اشاره کرد:

1. **ارزش‌گذاری بر اساس ارزش دفتری**: در این روش، ارزش دارایی بر اساس مبلغ ثبت‌شده در دفاتر مالی محاسبه می‌شود. این ارزش معمولاً شامل هزینه‌های اولیه خرید و استهلاک انباشته است.

2. **ارزش‌گذاری بر اساس ارزش بازار**: در این روش، ارزش دارایی بر اساس قیمتی که در بازار برای آن تعیین می‌شود، محاسبه می‌گردد. این ارزش می‌تواند تحت تأثیر عوامل مختلفی مانند عرضه و تقاضا قرار گیرد.

3. **ارزش‌گذاری بر اساس ارزش اقتصادی**: این روش به بررسی سودآوری و منافع اقتصادی که دارایی در آینده ایجاد می‌کند، می‌پردازد. ارزش اقتصادی معمولاً بر اساس جریان‌های نقدی آینده تخمین زده می‌شود.

4. **ارزش‌گذاری بر اساس معاملات


##Custom Embedding

In [ ]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import LangchainEmbedding, ServiceContext

# load in HF embedding model from langchain
embed_model = LangchainEmbedding(HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2'))
service_context = ServiceContext.from_defaults(embed_model=embed_model)

new_index = VectorStoreIndex.from_documents(
    documents,
    service_context=service_context,
)

In [ ]:
query_engine = new_index.as_query_engine()
response = query_engine.query("list 5 important points from this book")
print(response)


1. Building wealth and being happy are skills that can be learned.
2. The Almanack of Naval Ravikant is a collection of Naval's wisdom and experience from the last ten years.
3. This book provides insight into Naval's principles for building wealth and creating long-term happiness.
4. This book is available for free download in pdf and e-reader versions on Navalmanack.com.
5. Eric Jorgenson is a product strategist and writer who joined the founding team of Zaarly.


In [ ]:
query_engine = new_index.as_query_engine()
response = query_engine.query("what naval says about wealth creation")
print(response)


Naval Ravikant says that wealth creation is not a one-time thing, but a skill that needs to be learned. He suggests asking yourself if what you are doing is authentic to you and if you are productizing, scaling, and using labor, capital, code, or media to do so. He also states that money is a way to transfer wealth, and that wealth is assets that can earn while you sleep, such as businesses, factories, robots, computer programs, and even houses that can be rented out.
